In [1]:
#convert excel to all be of same data type
import pandas
import numpy
import math
import random

#can probably make this a def function
pathname = "/Users/chugh/Documents/UFC Fight Code/Fighter stats train new.csv"
pathname2 = "/Users/chugh/Documents/UFC Fight Code/Fighter stats train new 2.csv"
stats = pandas.read_csv(pathname, index_col = 0)
stats = stats.reset_index()
#print(stats)
stats2 = stats.astype('float32')
stats2.to_csv(pathname2, index=False)
stats3 = pandas.read_csv(pathname2, index_col = 0)

In [2]:
#want to write code that takes in two fighters stats and makes a prediction of who will win 
#additionally, would like to at some point make it so that it knows how they will win

#in order to do this, we will feed in data from both fighters as one data structure
#based on the features for each fighter, the algorithm will try to predict who will win

#we are going to want to create a structure of features and labels
#for features, we will include all data for Fighter 1 and Fighter 2 labeled F1 amd F2
#labels will contain the winner of each match (F1 or F2)

import os
import matplotlib.pyplot as plt
import tensorflow as tf
tf.enable_eager_execution()


#load in training data
pathname = "/Users/chugh/Documents/UFC Fight Code/Fighter stats train new 2.csv"

# column order in CSV file
# column_names = ['SSpM','StrDef','TDpM','TDDef','1stRnd','2ndRnd','3rdRnd','4thRnd','5thRnd','Dec','Rank','WL','FinUFC','Age','Dur','SSpM2','StrDef2','TDpM2','TDDef2','1stRnd2','2ndRnd2','3rdRnd2','4thRnd2','5thRnd2','Dec2','Rank2','WL2','FinUFC2','Age2','Dur2','Result']
column_names = ['SSpM','StrDef','TDpM','TDDef','Rank','WL','FinUFC','Age','Dur','SSpM2','StrDef2','TDpM2','TDDef2','Rank2','WL2','FinUFC2','Age2','Dur2','Result']


feature_names = column_names[:-1]
label_name = column_names[-1]

print("Features: {}".format(feature_names))
print("Label: {}".format(label_name))

class_names = ['Fighter 1', 'Fighter 2']

batch_size = 5

train_dataset = tf.data.experimental.make_csv_dataset(
    pathname,
    batch_size,
    column_names=column_names,
    label_name=label_name,
    num_epochs=1)

features, labels = next(iter(train_dataset))
print(features)

Features: ['SSpM', 'StrDef', 'TDpM', 'TDDef', 'Rank', 'WL', 'FinUFC', 'Age', 'Dur', 'SSpM2', 'StrDef2', 'TDpM2', 'TDDef2', 'Rank2', 'WL2', 'FinUFC2', 'Age2', 'Dur2']
Label: Result
Instructions for updating:
Use `tf.data.Dataset.interleave(map_func, cycle_length, block_length, num_parallel_calls=tf.data.experimental.AUTOTUNE)` instead. If sloppy execution is desired, use `tf.data.Options.experimental_determinstic`.
OrderedDict([('SSpM', <tf.Tensor: id=126, shape=(5,), dtype=float32, numpy=array([3.46, 5.6 , 3.32, 2.88, 4.03], dtype=float32)>), ('StrDef', <tf.Tensor: id=128, shape=(5,), dtype=float32, numpy=array([0.65, 0.53, 0.51, 0.47, 0.64], dtype=float32)>), ('TDpM', <tf.Tensor: id=132, shape=(5,), dtype=float32, numpy=array([0.35, 0.48, 1.  , 2.69, 4.5 ], dtype=float32)>), ('TDDef', <tf.Tensor: id=130, shape=(5,), dtype=float32, numpy=array([0.51, 0.7 , 0.45, 0.58, 0.85], dtype=float32)>), ('Rank', <tf.Tensor: id=124, shape=(5,), dtype=float32, numpy=array([10., 10., 55., 27., 38.],

In [3]:
def pack_features_vector(features, labels):

    features = tf.stack(list(features.values()), axis=1)

    return features, labels
train_dataset = train_dataset.map(pack_features_vector)

features, labels = next(iter(train_dataset))

print(features[:5])

tf.Tensor(
[[ 3.46        0.65        0.35        0.51       10.          0.5
  10.         42.          1.          3.32        0.63        1.13
   0.67        6.          0.6        10.         31.          0.71428573]
 [ 5.6         0.53        0.48        0.7        10.          0.8
   4.         33.          1.          2.3         0.53        3.28
   0.48        7.          0.6        10.         32.          0.5       ]
 [ 3.32        0.51        1.          0.45       55.          0.8
   6.         26.          1.          4.94        0.44        0.
   0.         56.          0.8         1.         33.          0.        ]
 [ 2.88        0.47        2.69        0.58       27.          0.5
  10.         32.          0.6         2.17        0.59        1.76
   0.77       93.          0.8         6.         33.          0.6666667 ]
 [ 4.03        0.64        4.5         0.85       38.          0.9
   1.         33.          1.          3.4         0.76        1.55
   1.         25

In [4]:
model = tf.keras.Sequential([
  tf.keras.layers.Dense(5, activation=tf.nn.relu, input_shape=(18,)),  # input shape required
  tf.keras.layers.Dense(5, activation=tf.nn.relu),
  tf.keras.layers.Dense(2)
])

In [5]:
predictions = model(features)
tf.nn.softmax(predictions[:5])

<tf.Tensor: id=284, shape=(5, 2), dtype=float32, numpy=
array([[0.0013013 , 0.9986987 ],
       [0.00391162, 0.9960884 ],
       [0.00350577, 0.99649423],
       [0.12710088, 0.8728992 ],
       [0.01958097, 0.980419  ]], dtype=float32)>

In [6]:
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
def loss(model, x, y, training):
  # training=training is needed only if there are layers with different
  # behavior during training versus inference (e.g. Dropout).
    y_ = model(x, training=training)
    return loss_object(y_true=y, y_pred=y_)


l = loss(model, features, labels, training=False)
print("Loss test: {}".format(l))

Loss test: 1.2009408473968506


In [7]:
def grad(model, inputs, targets):
    with tf.GradientTape() as tape:
        loss_value = loss(model, inputs, targets, training=True)

    return loss_value, tape.gradient(loss_value, model.trainable_variables)
optimizer = tf.keras.optimizers.SGD(learning_rate=0.01)
loss_value, grads = grad(model, features, labels)
print("Step: {}, Initial Loss: {}".format(optimizer.iterations.numpy(),loss_value.numpy()))

optimizer.apply_gradients(zip(grads, model.trainable_variables))

print("Step: {}, Loss: {}".format(optimizer.iterations.numpy(),loss(model, features, labels, training=True).numpy()))

Step: 0, Initial Loss: 1.2009408473968506
Step: 1, Loss: 0.9932059049606323


In [8]:
## Note: Rerunning this cell uses the same model variables

# Keep results for plotting
train_loss_results = []
train_accuracy_results = []

num_epochs = 201

for epoch in range(num_epochs):
    epoch_loss_avg = tf.keras.metrics.Mean()
    epoch_accuracy = tf.keras.metrics.SparseCategoricalAccuracy()

    # Training loop - using batches of 32
    for x, y in train_dataset:
        # Optimize the model
        loss_value, grads = grad(model, x, y)
        optimizer.apply_gradients(zip(grads, model.trainable_variables))

        # Track progress
        epoch_loss_avg.update_state(loss_value)  # Add current batch loss
        # Compare predicted label to actual label
        # training=True is needed only if there are layers with different
        # behavior during training versus inference (e.g. Dropout).
        epoch_accuracy.update_state(y, model(x, training=True))

      # End epoch
    train_loss_results.append(epoch_loss_avg.result())
    train_accuracy_results.append(epoch_accuracy.result())
    if epoch % 50 == 0:
        print("Epoch {:03d}: Loss: {:.3f}, Accuracy: {:.3%}".format(epoch,epoch_loss_avg.result(),epoch_accuracy.result()))

Epoch 000: Loss: 1.068, Accuracy: 72.000%
Epoch 050: Loss: 0.489, Accuracy: 92.000%
Epoch 100: Loss: 0.510, Accuracy: 86.000%
Epoch 150: Loss: 0.489, Accuracy: 82.000%
Epoch 200: Loss: 0.480, Accuracy: 86.000%


In [9]:
import pandas
import numpy
import math

pathname = "/Users/chugh/Documents/UFC Fight Code/Fighter stats test new.csv"
pathname2 = "/Users/chugh/Documents/UFC Fight Code/Fighter stats test new 2.csv"
stats = pandas.read_csv(pathname, index_col = 0)
stats = stats.reset_index()
stats2 = stats.astype('float32')
stats2.to_csv(pathname2, index=False)
stats3 = pandas.read_csv(pathname2, index_col = 0)

In [10]:
test_url = "/Users/chugh/Documents/UFC Fight Code/Fighter stats test new 2.csv"

batch_size = 2
test_dataset = tf.data.experimental.make_csv_dataset(
    test_url,
    batch_size,
    column_names=column_names,
    label_name='Result',
    num_epochs=1,
    shuffle=False)

test_dataset = test_dataset.map(pack_features_vector)

In [11]:
test_accuracy = tf.keras.metrics.Accuracy()

for (x, y) in test_dataset:
  # training=False is needed only if there are layers with different
  # behavior during training versus inference (e.g. Dropout).
    logits = model(x, training=False)
    prediction = tf.argmax(logits, axis=1, output_type=tf.int32)
    print(prediction)
    test_accuracy(prediction, y)

print("Test set accuracy: {:.3%}".format(test_accuracy.result()))

tf.Tensor([1 1], shape=(2,), dtype=int32)
tf.Tensor([1 0], shape=(2,), dtype=int32)
tf.Tensor([1 1], shape=(2,), dtype=int32)
tf.Tensor([0 1], shape=(2,), dtype=int32)
tf.Tensor([0 1], shape=(2,), dtype=int32)
tf.Tensor([0 1], shape=(2,), dtype=int32)
tf.Tensor([1 1], shape=(2,), dtype=int32)
tf.Tensor([1], shape=(1,), dtype=int32)
Test set accuracy: 60.000%


In [12]:
#for new data
import pandas
import numpy
import math

pathname = "/Users/chugh/Documents/UFC Fight Code/Fighter Stats 81920.csv"
pathname2 = "/Users/chugh/Documents/UFC Fight Code/Fighter Stats 81920 2.csv"
stats = pandas.read_csv(pathname, index_col = 0)
stats = stats.reset_index()
stats2 = stats.astype('float32')
stats2.to_csv(pathname2, index=False)

#will default results to 1
test_url = "/Users/chugh/Documents/UFC Fight Code/Fighter Stats 81920 2.csv"

batch_size = 8 #batch size should be equal to the number of fights
test_dataset = tf.data.experimental.make_csv_dataset(
    test_url,
    batch_size,
    column_names=column_names,
    label_name='Result',
    num_epochs=1,
    shuffle=False)

test_dataset = test_dataset.map(pack_features_vector)

test_accuracy = tf.keras.metrics.Accuracy()

for (x, y) in test_dataset:
  # training=False is needed only if there are layers with different
  # behavior during training versus inference (e.g. Dropout).
    logits = model(x, training=False)
    prediction = tf.argmax(logits, axis=1, output_type=tf.int32)
    print(prediction)
    test_accuracy(prediction, y)
    
tf.nn.softmax(logits)

tf.Tensor([1 1 0 0 1 0 1], shape=(7,), dtype=int32)


<tf.Tensor: id=222370, shape=(7, 2), dtype=float32, numpy=
array([[0.00282314, 0.9971769 ],
       [0.11440118, 0.8855988 ],
       [0.5993785 , 0.40062144],
       [0.628533  , 0.37146696],
       [0.45969424, 0.5403057 ],
       [0.6273176 , 0.37268242],
       [0.04103351, 0.95896643]], dtype=float32)>

In [ ]:
model.save("/Users/chugh/Documents/UFC Fight Code/model_8320_hundred")

In [ ]:
print(column_names)